In [1]:
import pandas as pd
import numpy as np
import networkx as nx

# Mutual Nearest Neighbour

In [2]:
def mnvForNodes(u, v, knnGraph):
    listU = knnGraph(u)
    listV = knnGraph(v)
    
    rankOfVinU = 100
    rankOfUinV = 100
    
    for i in range(len(listU)):
        if listU[i]==v:
            rankOfVinU = i+1
            break
    
    for i in range(len(listU)):
        if listV[i]==u:
            rankOfUinV = i+1
            break

    return rankOfUinV+rankOfVinU

In [5]:
def mnvForClusters(DictofClusters, c1, c2, knnGraph):
    cluster1 = DictofClusters[c1]
    cluster2 = DictofClusters[c2]
    
    s = 0
    
    for u in cluster1:
        for v in cluster2:
            val = mnvForNodes(u, v, knnGraph)
            s = s + val
    length = len(cluster1) * len(cluster2)
    
    return s/length

In [6]:
def mutual_NN_neighbour(DictofClusters, knnGraph, numofClusters, maxMNV):
    while len(DictofClusters) > numofClusters:
        minVal = 200
        clusterToUnion1 = DictofClusters[0]
        clusterToUnion2 = DictofClusters[1]
        for c1 in DictofClusters:
            for c2 in DictofClusters:
                currVal = mnv(DictofClusters, c1, c2, knnGraph)
                if (c1 != c2) and (currVal < minVal):
                    minVal = currVal
                    clusterToUnion1 = c1
                    clusterToUnion2 = c2
        if minVal > maxMNV:
            break
        for v in clusterToUnion2:
            DictofClusters[clusterToUnion1].append(v)
        del DictofClusters[clusterToUnion2]
        
    return DictofClusters
        

In [9]:
Dict = {}
nodeSize = 32449
for i in range(nodeSize):
    Dict[i] = [i]
print(len(Dict))

32449
